<a href="https://colab.research.google.com/github/mantoan-thi/Aprendizagem-por-Refor-o-com-Augmented-Random-Search-ARS-/blob/main/Aprendizagem_por_Refor%C3%A7o_com_Augmented_Random_Search_(ARS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import os
import numpy as np
import gym
from gym import wrappers

class Hp():
  def __init__(self):
    self.nb_steps = 1000         # número de loops que vamos utilizar para o treinamento.
    self.episode_lenght = 1000   # máxima duração de um epsódio.
    self.learning_rate = 0.002   # Taxa de aprendizagem.
    self.nb_directions = 16      # número de direções -  quantidade de matrizes...
    self.nb_best_directions = 16 # seleciona as melhores matrizes.
    assert self. nb_best_directions <= self.nb_directions # nb_best_directions tem que ser menor ou igual ao self.nb_directions.
    self.noise = 0.03            # Usado para não termos variância nos dados.
    self.seed = 1                # usamos para termos os mesmos resultados.
    self.env_name = ''           # nome do ambiente.

class Normalizer():
  def __init__(self,nb_inputs):
    self.n = np.zeros(np_inputs)          # Quantidade de entradas.
    self.mean = np.zeros(np_inputs)       # Média
    self.mean_diff = np.zeros(np_inputs)  # 
    self.var = np.zeros(np_inputs)        # Variância
  def observe(self,x):
    self.n +=1.
    last_mean = self.mean.copy()
    self.mean += (x-self.mean)/self.n
    self.mean_diff += (x -last_mean)*(x - self.mean)
    self.var = (self.mean_diff/self.n).clip(min = 1e-2) #0.001
  def normalizer(self,inputs):
    obs_mean = self.mean
    obs_std = np.sqrt(self.var)
    return (inputs -obs_mean)/obs_std

class Policy():
  def __init__(self,input_size,output_size):
    self.theta = np.zeros((output_size,input_size)) # serve para atualizar os pesos.

  def evaluate (self,input,delta=None,direction = None):
    if direction is None:
      return self.theta.dot(input)
    elif direction == 'positive':
      return (self.theta +hp.noise * delta).dot(input)
    else:
      return (self.theta -hp.noise * delta).dot(input)
  def sample_deltas(self):
    return [np.random.randn(*self.theta.shape) for _ in range(Hp.nb_directions)]

  # Atualização dos pesos
  def update(self,rollouts, sigma_r):
    step = np.zeros(self.theta.shape)
    for r_pos, r_neg, d in rollouts:
      step +=(r_pos - r_neg)*d
    self.theta +=Hp.learning_rate/(Hp.nb_best_directions*sigma_r)*step

# Explora a política em uma direção específica e dentro de um epsódio
def explore(env,normalizer,policy,direction = None, delta = None):
  state = env.reset()
  done = False
  num_plays = 0.
  sum_rewards = 0
  while not done and num_plays < Hp.episode_lenght:
    normalizer.observe(state)
    state = normalizer.normalizer(state)
    action = policy.evaluate(state,delta,direction)
    state,rewards,done,_ = env.step(action)
    reward = max(min(reward,),-1)
    sum_rewards +=reward
    num_plays +=1
  return sum_rewards

# Treinamento da inteligência artifical
def train(env,policy,normalizer,Hp):
  for step in range(Hp.nb_steps):
    # Inicialização das pertubações (delta) e as recompensas positivas e negativas
    delta = policy.sample_deltas()
    positive_rewards = [0]*Hp.np_directions
    negative_rewards = [0]*Hp.np_directions
    # Obtenção da recompensas das direções positivas.
    for k in range(Hp.nb_directions):
      positive_rewards[k] = explore(env,normalizer,direction='positive',delt=delta[k])
    # Obtenção da recompensas das direções negativas.
    for k in range(Hp.nb_directions):
      negative_rewards[k] = explore(env,normalizer,direction='negative',delt=delta[k])
    # Obtendo todas as recompensas positivas e negativas para computar o desvio padrão dessas recompensas
    all_rewards = np.array(positive_rewards + negative_rewards)
    sigma_r = all_rewards.std()
    # Ordenação dos dollouts e seleção das melhrores direções
    scores ={k:max(r_pos,r_neg) for k,(r_pos,r_neg) in enumerate(zip(positive_rewards,negative_rewards))}
    order = sorted(scores.keys(),key=lambda x: scores[x],reverse=True)[:Hp.nb_best_directions]
    rollouts = [(positive_rewards[k],negative_rewards[k],delta[k]) for k in order]
    # Atualização da política
    policy.update(rollouts,sigma_r)
    # impressão da recompensa final depois da atualização
    reward_evaluation = explore(env,normalizer,policy)
    print('Step',step,'Reward', reward_evaluation)

In [33]:
def mkdir(base,name):
  path = os.path.join(base,name)
  if not os.path.exists(path):
    os.makedirs(path)
  return path
work_dir = mkdir('exp','brs')
monitor_dir = mkdir(work_dir,'monitor')

In [35]:
hp = Hp()
np.random.seed(hp.seed)
env = gym.make(hp.env_name)
env = wrapper.Monitor(env,monitor_dir,force = True)
nb_inputs = env.observation_space.shape[0]
nb_outputs = env.action_space.shape[0]
policy = Policy.(nb_inputs,nb_outputs)
normalizer = Normalizer(nb_inputs)
train(env,policy,normalizer,hp)

SyntaxError: ignored